In [4]:
from table_builder import *
from class_children import *
import math
from scipy.stats import norm
from scipy.stats import logistic
from scipy.stats import bernoulli
from scipy import special
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from debug import ipsh
import warnings
warnings.filterwarnings('ignore')
import sys
sys.setrecursionlimit(5000) #change the default recursion depth

## Create Nash Child Class

As it is, $\theta = (a_R,a_L,a_C,b,c,\mu,s,p_C,\sigma,\epsilon)$.

In [5]:
class Nash(Bank):
    '''
    This class is a child of the Bank class defined in the class_children script.
    
    It provides methods for variable definition, recursively solving for rho and zeta, implementing gradient ascent,
        and tests to ensure that the code is functioning properly
    '''

    # ideal interest rate
    def I(self,a,b,c):
        return a+b*self.table['Inflation']+c*self.table['OutGap']
    
    # left and right extreme constants
    def add_ZLZR(self,theta):
        
        aR=theta[0]; aL=theta[1]; aC=theta[2]; b=theta[3]; c=theta[4]

        i = self.table['PriorTarget'].copy()
        self.table['zL'] = i-self.I(aL,b,c)
        self.table['zR'] = i-self.I(aR,b,c)
    
    # left and right extreme ideal interest rates
    def add_ILIR(self,theta,zeta='zeta'):
        
        aR=theta[0]; aL=theta[1]; aC=theta[2]; b=theta[3]; c=theta[4]
        
        self.table['IL_'+zeta] = self.I(aL,b,c)+self.table[zeta]
        self.table['IR_'+zeta] = self.I(aR,b,c)+self.table[zeta]
    
    # approximated indicator function
    def G(self,theta,step_approx=norm.cdf,zeta='zeta',target='Target'):
        
        aR=theta[0]; aL=theta[1]; aC=theta[2]; b=theta[3]; c=theta[4];
        mu=theta[5]; s=theta[6]; pC=theta[7]; sigma=theta[8]; eps=theta[9]
        
        self.table['G'] = step_approx((self.I(aC,b,c)-self.table[target]+self.table[zeta])/eps)
        
        return self.table['G']
    
    # derivative of G
    def g(self,theta,zeta='zeta',target='Target',step_approx_PDF=norm.pdf):

        aR=theta[0]; aL=theta[1]; aC=theta[2]; b=theta[3]; c=theta[4];
        mu=theta[5]; s=theta[6]; pC=theta[7]; sigma=theta[8]; eps=theta[9]
        
        self.table['g'] = step_approx_PDF((self.I(aC,b,c)-self.table['Target']+self.table[zeta])/eps)
        
        return self.table['g']
    
    # the next four functions recursively solve for rho and zeta
    # first, make an initial guess for rho
    def guess_rho(self,theta,zeta='zeta',target='target'):
        
        aR=theta[0]; aL=theta[1]; aC=theta[2]; b=theta[3]; c=theta[4];
        mu=theta[5]; s=theta[6]; pC=theta[7]; sigma=theta[8]; eps=theta[9]
        
        self.add_ZLZR(theta)
        rho = pd.Series([1-self.F(mu)]*self.table['Date'].size)
        self.table['rho_'+zeta] = rho.copy()

        return rho
    
    # use rho to solve for zeta
    def give_z(self,theta,plot=False,zeta='zeta',target='Target',rho='rho_zeta',chairman=False):
        
        aR=theta[0]; aL=theta[1]; aC=theta[2]; b=theta[3]; c=theta[4];
        mu=theta[5]; s=theta[6]; pC=theta[7]; sigma=theta[8]; eps=theta[9]
        
        self.table['ChangeSign'] = (self.table[target]-self.table['PriorTarget']).apply(np.sign)
        comm = 1/2*(self.table[target]-self.table['PriorTarget'])
        i = self.table[target].copy()
      
        B = 2*((self.I(aL,b,c)+self.I(aR,b,c))/2-i+1/2*comm)
        C = (self.I(aL,b,c)-i)*(self.I(aR,b,c)-i)+comm*(self.table[rho]*\
                    self.I(aL,b,c)+(1-self.table[rho])*self.I(aR,b,c)-i)

        if plot:
            self.add_ZLZR(theta)
            x = np.arange(-2,5,.1)
            y = x**2+B[1]*x+C[1]
            zero = np.zeros(x.size)
            fig, ax = plt.subplots()
            ax.plot(x,y)
            ax.plot(x,zero)
            ax.axvline(self.table['zL'][1],color='r')
            ax.axvline(self.table['zR'][1],color='m')
            plt.show()
        
        self.table[zeta] = pd.Series([0]*self.T)
        self.table[zeta][self.table['ChangeSign']==1] = (-B+(B**2-4*C)**(1/2))/2
        self.table[zeta][self.table['ChangeSign']==-1] = (-B-(B**2-4*C)**(1/2))/2
        return self.table[zeta]

    # use zeta to solve for rho  
    def give_rho(self,theta,zeta='zeta',target='Target',new=False,F=logistic.cdf):
        
        aR=theta[0]; aL=theta[1]; aC=theta[2]; b=theta[3]; c=theta[4];
        mu=theta[5]; s=theta[6]; pC=theta[7]; sigma=theta[8]; eps=theta[9]
        
        i = self.table[target].copy()
        
        if new:
            rho = pC*self.G(theta,zeta=zeta)+(1-pC)*((self.I(mu,b,c)+self.table['new_'+zeta]-i)/s).apply(F)
            rho[rho>1]=1; rho[rho<0]=0
            self.table['newRho_'+zeta] = rho.copy()
            
        else:
            rho = pC*self.G(theta,zeta=zeta)+(1-pC)*((self.I(mu,b,c)+self.table[zeta]-i)/s).apply(F)
            rho[rho>1]=1; rho[rho<0]=0
            self.table['rho_'+zeta] = rho.copy()   
        
        return rho
    
    # repeat until both rho and zeta converge
    def get_rho_z_helper(self,theta,zeta='zeta',target='Target',conv=0.0001,print_n=False,n=0):
        
        aR=theta[0]; aL=theta[1]; aC=theta[2]; b=theta[3]; c=theta[4];
        mu=theta[5]; s=theta[6]; pC=theta[7]; sigma=theta[8]; eps=theta[9]
        
        n+=1
        
        newZeta = 'new_'+zeta
        self.give_z(theta,zeta=newZeta,target=target,rho='rho_'+zeta)
        self.give_rho(theta,zeta=zeta,target=target,new=True)
        
        if n==100:
            print('too long')
            ipsh()
        
        if target!='Target':
            rho_conv = all((self.table['rho_'+zeta]-self.table['newRho_'+zeta])\
                      [(self.table['Chairman_'+zeta]==0)].apply(abs)<conv)
            z_conv = all((self.table[zeta]-self.table[newZeta])\
                    [(self.table['Chairman_'+zeta]==0)].apply(abs)<conv)
            
        else:                             
            rho_conv = all((self.table['rho_'+zeta]-self.table['newRho_'+zeta])\
                          [(self.table['ChangeSign']!=0)&(self.table['Chairman_'+zeta]==0)].apply(abs)<conv)
            z_conv = all((self.table[zeta]-self.table[newZeta])\
                        [(self.table['ChangeSign']!=0)&(self.table['Chairman_'+zeta]==0)].apply(abs)<conv)
            
        if n==100:
            ipsh()
            
        self.table[zeta] = self.table[newZeta].copy(); self.table = self.table.drop([newZeta],axis=1)
        self.table['rho_'+zeta] = self.table['newRho_'+zeta].copy(); self.table = self.table.drop(['newRho_'+zeta],axis=1) 
        
        try:
            self.table[zeta][self.table['Chairman_'+zeta]==1]=\
            -(aC+b*self.table['Inflation']+c*self.table['OutGap']-self.table[target])
        except:
            pass
       
        if (rho_conv,z_conv) == (True,True):
            if print_n:
                print(n)
            return self.table
            
        return self.get_rho_z_helper(theta,zeta,target,conv,print_n,n)
    
    # chairmain indicator function, which chairman induced preferences
    def chairman_helper(self,theta,zeta='zeta'):
        
        aR=theta[0]; aL=theta[1]; aC=theta[2]; b=theta[3]; c=theta[4];
        mu=theta[5]; s=theta[6]; pC=theta[7]; sigma=theta[8]; eps=theta[9]
        
        self.table['Chairman_'+zeta] = pd.Series([0]*self.T)
        self.table['lower'] = pd.Series([(1-pC)*self.F((mu-aC)/s)]*self.T)
        self.table['upper'] = pd.Series([pC+(1-pC)*self.F((mu-aC)/s)]*self.T)
        self.give_z(theta,zeta='lowerZ',rho='lower')
        self.give_z(theta,zeta='upperZ',rho='upper')
        self.table['atLower'] = (self.table['Target']-b*self.table['Inflation']-c*self.table['OutGap']-self.table['lowerZ']).copy()       
        self.table['atUpper'] = (self.table['Target']-b*self.table['Inflation']-c*self.table['OutGap']-self.table['upperZ']).copy()
        self.dropCols(['lower','upper','lowerZ','upperZ'])
        
        try:
            self.table['Chairman_'+zeta][(aC<self.table['atUpper']) & (aC>self.table['atLower'])]=1
        except:
            pass
    
    # the next two functions implement a randomized grid search to ensure that the gradient ascent doesn't get stuck at local optima
    def make_grid(self,theta,bin_size=2,pC_bin_size=0.4,num_draws=40):

        aR=theta[0]; aL=theta[1]; aC=theta[2]; b=theta[3]; c=theta[4];
        mu=theta[5]; s=theta[6]; pC=theta[7]; sigma=theta[8]; eps=theta[9]
        
        params = np.array([aR,aL,aC,b,c,mu,s,pC,sigma])
        grids = []
        for i in range(len(theta)-1):
            grids.append([])
            for j in range(num_draws):
                if i==6 or i==8:
                    grids[i].append(random.uniform(max(params[i]-bin_size,0),params[i]+bin_size))
                elif i==7:
                    grids[i].append(random.uniform(max(params[i]-pC_bin_size,0),min(params[i]+pC_bin_size,1)))
                else:
                    grids[i].append(random.uniform(params[i]-bin_size,params[i]+bin_size))
            
        self.grid = pd.DataFrame(np.transpose(np.array(grids)))
        self.grid.columns = ['aR','aL','aC','b','c','mu','s','pC','sigma']
        return(self.grid)
    
    def compute_grid_ll(self,theta,**grid_kwargs):

        self.make_grid(theta,**grid_kwargs)
        params = {0:'aR',1:'aL',2:'aC',3:'b',4:'c',5:'mu',6:'s',7:'pC',8:'sigma'}
        
        def make_theta_ll(row):
            rowTheta = row.values
            rowTheta = np.append(rowTheta,theta[-1])
            try:
                return self.lln(rowTheta)
            except:
                return None
                
        self.grid['LL'] = self.grid.apply(lambda row: make_theta_ll(row), axis=1)
        return(self.grid)
    
    # ensure that variance is not less than 0 and that the chairman's ideal policy is between 0 and 1
    def check_params(self,theta):
        
        if theta[6]<0:
            theta[6]=0
        
        if theta[7]>1:
            theta[7]=1
            
        elif theta[7]<0:
            theta[7]=0
            
        if theta[8]<0:
            theta[8]=0
            
        return theta
    
    # if desired, weights to create incrementally decreasing intervals for grid search
    def search_weights(self,n):
        
        if n<=20:
            return 1
        elif n<=30:
            return 0.5
        elif n<=40:
            return 0.25
        elif n<=60:
            return 0.125
        elif n<=80:
            return 0
    
    # implement gradient until convergence
    def grad_ascent_nash(self,alpha,estimate,conv = 0.0001,ll=False,est=False,n=1,m=0,v=0,beta_1=0.9,beta_2=0.999,**grid_kwargs):
        
        anotherEpsilon = 10e-8
        estimate=self.check_params(estimate)
        
        #num_draws=10,bin_size=2*random.uniform(.25,1),pC_bin_size=0.4*random.uniform(0.25,1)

        if n%10==1 and n<70:
            self.compute_grid_ll(estimate,**grid_kwargs)
            row = self.grid['LL'].argmax()
            newEstimate = self.grid.loc[row].copy().values
            newEstimate[-1] = estimate[-1]
            if self.grid['LL'].max()>self.lln(estimate):
                estimate = newEstimate
        
        if est:
            print(estimate)
    
        
        #if (BoC.table['zetaHigh']-BoC.table['zetaLow']<0).all() is not False:
            #print("zetaHigh is greater than zetaLow")    #check to see if zetaHigh is greater than zetaLow in the discrete case
        
        
        
        
        # adapted from https://towardsdatascience.com/adam-latest-trends-in-deep-learning-optimization-6be9a291375c
        gradient = self.grad_nash(estimate)
        m = beta_1 * m + (1 - beta_1) * gradient
        m_tild = beta_1*m + (1-beta_1)*gradient
        v = beta_2 * v + (1 - beta_2) * np.power(gradient, 2)
        m_hat = m_tild / (1 - np.power(beta_1, n))
        v_hat = v / (1 - np.power(beta_2, n))
        w = alpha*m_hat / (np.sqrt(v_hat) + anotherEpsilon)

        update = estimate[0:-1]+w
        update = np.append(update,estimate[-1])
        
        update = self.check_params(update)
        
        theta=self.check_params(estimate)

        priorLL = self.lln(estimate)
        newLL = self.lln(update)

        improve = newLL>priorLL

        n+=1

        if ll:
            print(newLL)

        if improve:    
            if abs(newLL-priorLL)<=conv:
                return update

        return(self.grad_ascent_nash(alpha,update,conv,ll,est,n,m,v))

        #return self.grad_ascent_nash(worseWeight*alpha,estimate,conv,ll,est,worseWeight,betterWeight,n,m,v)
            
    # check to ensure that the hard coded partial derivates are equal to those achieved by incrementally changing the log likelihood function
    def compare_grad_to_lln(self,theta,epsilon=0.00001,delta=0.001,start=0,end=9):
        
        params = {0:'aR',1:'aL',2:'aC',3:'b',4:'c',5:'mu',6:'s',7:'pC',8:'sigma'}
        
        good = True
        grad = self.grad_nash(theta)
        
        for i in range(start,end):
            beta = theta[:]
            beta[i] = beta[i]+epsilon
            
            if abs(self.grad_nash(theta)[i]-(self.lln(beta)-self.lln(theta))/epsilon)>=delta:
                print(params[i])
                print(grad[i])
                print(((self.lln(beta)-self.lln(theta))/epsilon))
                good = False
                
        print('Done!')       
        if good:
            print("You're all good!")
            
    # ensure that following the gradient improves the likelihood function
    def check_grad_improving_nash(self,theta,epsilon=0.00001,start=0,end=9):
        
        good = True
        llh = self.lln(theta)
        
        for i in range(start,end):
            mutable_theta = theta[:]
            mutable_theta[i] = mutable_theta[i]+epsilon*self.grad_nash(theta)[i]
            if (self.lln(mutable_theta)<llh):
                good = False
                print(i)
        
        print('Done!')       
        if good:
            print("You're all good!")
            
    
    # the following two methods are for simulation and still incomplete        
    def all_estimates(self,theta,samples=10,increment=0.001):
        
        all_zeta = norm.rvs(loc=theta[-4], scale=theta[-3], size=samples)
        
        hessians = []
        for zetas in all_zeta:
            hessians.append(self.single_estimate(theta,zetas,increment))
            
        return(hessians)
            
        
    def single_estimate(self,theta,z,increment):
            
        weights = 2*((bernoulli.rvs(0.5,size=len(theta)))-0.5) * increment
        
        theta_lower = theta-weights
        theta_upper = theta-weights
        
        grad_lower = np.reshape(self.grad_nash(theta_lower,mc_z=z),(len(self.grad_nash(theta_lower,mc_z=z)),1))
        grad_upper = np.reshape(self.grad_nash(theta_upper,mc_z=z),(len(self.grad_nash(theta_upper,mc_z=z)),1))
        
        reg_1 = self.table.where(self.table['ChangeSign']==1).copy()
        reg_2 = self.table.where(self.table['ChangeSign']==0).copy()
        reg_3 = self.table.where(self.table['ChangeSign']==-1).copy()
        
        ipsh()
        
        estimate = 0.5 * (np.matmul((grad_upper/2),(1/weights)) + 
                          np.transpose(np.matmul((grad_lower/2),(1/weights)))) 
        return(estimate)
                            
            
Fed = Nash('Fed2',trunc=(7,None),dropCols=['Change','Tau'])

In [6]:
BoC.grad_nash(theta,mc_z=0.5)

NameError: name 'BoC' is not defined

### Create objects for the cases of a continuum of players and a discrete number of players

In [7]:
class Continuous(Nash):
    '''
    This class creates objects for the case of a continuum of players. It has methods to compute log likelihood its gradient.
    '''
    
    # implement the methods of the Nash object to get rho and zeta
    def get_rho_z(self,theta,**helper_kwargs):
        
        aR=theta[0]; aL=theta[1]; aC=theta[2]; b=theta[3]; c=theta[4];
        mu=theta[5]; s=theta[6]; pC=theta[7]; sigma=theta[8]; eps=theta[9]
                      
        self.guess_rho(theta)
        self.give_z(theta)
        self.give_rho(theta)
        self.chairman_helper(theta)
        self.get_rho_z_helper(theta,**helper_kwargs) 
        self.add_ZLZR(theta)
        self.add_ILIR(theta)
        
        try:
            self.table['zeta'][self.table['ChangeSign']==0]=None; self.table['rho_zeta'][self.table['ChangeSign']==0]=None
            self.table['rho_zeta'][self.table['Chairman_zeta']==1]=None
        except:
            pass
            
        self.dropCols(['atLower','atUpper'])
        
        return self.table
    
    # evaluate the log likelihood function
    def lln(self,theta):
        
        aR=theta[0]; aL=theta[1]; aC=theta[2]; b=theta[3]; c=theta[4];
        mu=theta[5]; s=theta[6]; pC=theta[7]; sigma=theta[8]; eps=theta[9]
        
        self.get_rho_z(theta)

        sumOne = -1/(2*sigma**2)*(self.table['zeta'].where(self.table['ChangeSign']!=0)**2).sum()
        sumTwo =  ((self.table['zR']/sigma).where(self.table['ChangeSign']==0).apply(norm.cdf)\
                -(self.table['zL']/sigma).where(self.table['ChangeSign']==0).apply(norm.cdf)).apply(math.log).sum()

        return (-(self.table['ChangeSign']!=0).size*(1/2*math.log(2*math.pi))+self.T*math.log(sigma)+sumOne+sumTwo)

    # since the log likelihood function is a sum of three items, we take the gradient of each and sum them together
    def grad_nash_first(self,theta): 
        
        aR=theta[0]; aL=theta[1]; aC=theta[2]; b=theta[3]; c=theta[4];
        mu=theta[5]; s=theta[6]; pC=theta[7]; sigma=theta[8]; eps=theta[9]
        
        i = self.table['Target'].copy(); z = self.table['zeta'].copy(); rho = self.table['rho_zeta'].copy()

        comm = 1/2*(self.table['Target']-self.table['PriorTarget'])
        den = 2*(z+(self.I(aL,b,c,)+self.I(aR,b,c))/2-i)+\
                comm+comm*(aL-aR)*(pC/eps*self.g(theta)+(1-pC)/s*self.f((self.I(mu,b,c)+z-i)/s))

        dZdaR = -(z+(self.I(aL,b,c)-i)+comm*(1-rho))/den
        dZdaL = -(z+(self.I(aR,b,c)-i)+comm*rho)/den
        dZdaC = -pC/eps*self.g(theta)*comm*(aL-aR)*(-1)
        dZdMu = -(1-pC)/s*comm*(aL-aR)*self.f((self.I(mu,b,c)+z-i)/s)/den
        dZdS = (comm*(1-pC)*(aL-aR)*((self.I(mu,b,c)+z-i)/s**2)*\
                self.f((self.I(mu,b,c)+z-i)/s))/den
        dZdpC = -comm*(aL-aR)*(self.G(theta)-self.F((self.I(mu,b,c)+z-i)/s))/den

        gradaR = -(z*dZdaR)/sigma**2
        gradaL = -(z*dZdaL)/sigma**2
        gradaC = -(z*dZdaC)/sigma**2
        gradB = (z*self.table['Inflation'])/sigma**2
        gradC = (z*self.table['OutGap'])/sigma**2
        gradMu = -(z*dZdMu)/sigma**2
        gradS = -(z*dZdS)/sigma**2
        gradpC = -(z*dZdpC)/sigma**2
        gradSigma = (z**2)/sigma**3

        return(gradaR,gradaL,gradaC,gradB,gradC,gradMu,gradS,gradpC,gradSigma)

    def grad_nash_second(self,theta):
        
        aR=theta[0]; aL=theta[1]; aC=theta[2]; b=theta[3]; c=theta[4];
        mu=theta[5]; s=theta[6]; pC=theta[7]; sigma=theta[8]; eps=theta[9]
        
        numr = (self.table['zR']/sigma).apply(norm.pdf)-(self.table['zL']/sigma).apply(norm.pdf)
        den = (self.table['zR']/sigma).apply(norm.cdf)-(self.table['zL']/sigma).apply(norm.cdf)
        
        gradaR = -1/sigma*((self.table['zR']/sigma).apply(norm.pdf)/den)
        gradaL = 1/sigma*((self.table['zL']/sigma).apply(norm.pdf)/den)
        gradaC = pd.Series([0]*self.T)
        gradB = -1/sigma*(self.table['Inflation']*numr/den)
        gradC = -1/sigma*(self.table['OutGap']*numr/den)
        gradMu = pd.Series([0]*self.T)
        gradS = pd.Series([0]*self.T)
        gradpC = pd.Series([0]*self.T)
        gradSigma = -1/(sigma**2)*(self.table['zR']*(self.table['zR']/sigma).apply(norm.pdf)-\
                                 self.table['zL']*(self.table['zL']/sigma).apply(norm.pdf))/den

        return(gradaR,gradaL,gradaC,gradB,gradC,gradMu,gradS,gradpC,gradSigma)
    
    def grad_nash_third(self,theta):
        
        i = self.table['Target'].copy(); z = self.table['zeta'].copy(); rho = self.table['rho_zeta'].copy()
        
        aR=theta[0]; aL=theta[1]; aC=theta[2]; b=theta[3]; c=theta[4];
        mu=theta[5]; s=theta[6]; pC=theta[7]; sigma=theta[8]; eps=theta[9]
        
        gradaR = pd.Series([0]*self.T)
        gradaL = pd.Series([0]*self.T)
        gradaC = z/sigma**2
        gradB = (z*self.table['Inflation'])/sigma**2
        gradC = (z*self.table['OutGap'])/sigma**2
        gradMu = pd.Series([0]*self.T)
        gradS = pd.Series([0]*self.T)
        gradpC = pd.Series([0]*self.T)
        gradSigma = (z**2)/sigma**3
        
        return(gradaR,gradaL,gradaC,gradB,gradC,gradMu,gradS,gradpC,gradSigma)
    
    # combined gradient
    def grad_nash(self,theta,mc_z=None,**get_rho_z_kwargs):
        
        sigma = theta[-2]
        if mc_z is None:
            self.get_rho_z(theta,**get_rho_z_kwargs)
        else:
            self.table['zeta'] = [mc_z]*self.T
            self.give_rho(theta)
            self.chairman_helper(theta)
            self.add_ZLZR(theta)
            self.add_ILIR(theta)
        
        gradaR=0.0; gradaL=0.0; gradaC=0.0; gradB=0.0; gradC=0.0; gradMu=0.0; gradS=0.0; gradpC=0.0; gradSigma=0.0
        grads = np.array([gradaR, gradaL, gradaC, gradB, gradC, gradMu, gradS, gradpC, gradSigma])
        for i in range(len(theta)-1):
            
            grads[i] = self.grad_nash_first(theta)[i].where(self.table['ChangeSign']!=0).where(self.table['Chairman_zeta']==0).sum()+\
                     self.grad_nash_second(theta)[i].where(self.table['ChangeSign']==0).sum()+\
                     self.grad_nash_third(theta)[i].where(self.table['ChangeSign']!=0).where(self.table['Chairman_zeta']==1).sum()
        
        grads[8]=grads[8]-self.T/sigma
        
        return(grads)

BoC = Continuous('BoC',cpiType='TotalCPI',lam=129600,trunc=(12,None),dropCols=['TotalCPI','TotalCPI_SA','Change','Tau'])

In [8]:
class Discrete(Nash):
    '''
    This class creates objects for the case of a discrete number of players. It has methods to compute log likelihood its gradient.
    '''

    # implement the methods of the Nash object to get rho and zeta
    def get_rho_z(self,theta,**helper_kwargs):
        
        aR=theta[0]; aL=theta[1]; aC=theta[2]; b=theta[3]; c=theta[4];
        mu=theta[5]; s=theta[6]; pC=theta[7]; sigma=theta[8]; eps=theta[9]
        
        self.table['T_high'] = self.table['Target']+0.125
        self.table['T_low'] = self.table['Target']-0.125
        self.table['ChangeSign']
        
        self.guess_rho(theta,zeta='zetaHigh')
        self.give_z(theta,zeta='zetaHigh',target='T_high',rho='rho_zetaHigh')
        self.give_rho(theta,zeta='zetaHigh',target='T_high')
        
        self.guess_rho(theta,zeta='zetaLow')
        self.give_z(theta,zeta='zetaLow',target='T_low',rho='rho_zetaLow')
        self.give_rho(theta,zeta='zetaLow',target='T_low')
        
        self.chairman_helper(theta,zeta='zetaHigh')
        self.chairman_helper(theta,zeta='zetaLow')
        
        self.get_rho_z_helper(theta,zeta='zetaHigh',target='T_high',**helper_kwargs) 
        self.get_rho_z_helper(theta,zeta='zetaLow',target='T_low',**helper_kwargs) 

        self.add_ZLZR(theta)
        self.add_ILIR(theta,zeta='zetaHigh')
        self.add_ILIR(theta,zeta='zetaLow')
        
        try:
            self.table['rho_zetaHigh'][self.table['Chairman_zetaHigh']==1]=None
            self.table['rho_zetaLow'][self.table['Chairman_zetaLow']==1]=None

        except:
            pass
            
        self.dropCols(['atLower','atUpper'])
    
    # evaluate the log likelihood function
    def lln(self,theta):
        
        aR=theta[0]; aL=theta[1]; aC=theta[2]; b=theta[3]; c=theta[4];
        mu=theta[5]; s=theta[6]; pC=theta[7]; sigma=theta[8]; eps=theta[9]
    
        self.get_rho_z(theta)
        
        llh =  ((self.table['zetaHigh']/sigma).apply(norm.cdf)\
                -(self.table['zetaLow']/sigma).apply(norm.cdf)).apply(math.log).sum()

        return(llh)

    # since this case has a much simpler gradient than the continuous case, we only combine two terms for the gradient
    def dZ(self,theta,target,zeta): 
        
        aR=theta[0]; aL=theta[1]; aC=theta[2]; b=theta[3]; c=theta[4];
        mu=theta[5]; s=theta[6]; pC=theta[7]; sigma=theta[8]; eps=theta[9]
        
        i = self.table[target].copy(); z = self.table[zeta].copy(); rho = self.table['rho_'+zeta].copy()

        comm = 1/2*(self.table[target]-self.table['PriorTarget'])
        den = 2*(z+(self.I(aL,b,c,)+self.I(aR,b,c))/2-i)+\
                comm+comm*(aL-aR)*(pC/eps*self.g(theta,zeta=zeta)+(1-pC)/s*self.f((self.I(mu,b,c)+z-i)/s))

        dZdaR = -(z+(self.I(aL,b,c)-i)+comm*(1-rho))/den
        dZdaL = -(z+(self.I(aR,b,c)-i)+comm*rho)/den
        dZdaC = -pC/eps*self.g(theta,zeta=zeta)*comm*(aL-aR)*(-1)
        dZdB = -self.table['Inflation']
        dZdC = -self.table['OutGap']
        dZdMu = -(1-pC)/s*comm*(aL-aR)*self.f((self.I(mu,b,c)+z-i)/s)/den
        dZdS = (comm*(1-pC)*(aL-aR)*((self.I(mu,b,c)+z-i)/s**2)*\
                self.f((self.I(mu,b,c)+z-i)/s))/den
        dZdpC = -comm*(aL-aR)*(self.G(theta,zeta=zeta)-self.F((self.I(mu,b,c)+z-i)/s))/den
        
        dZdaR[self.table['Chairman_'+zeta]==1]=0
        dZdaL[self.table['Chairman_'+zeta]==1]=0
        dZdaC[self.table['Chairman_'+zeta]==1]=-1
        dZdMu[self.table['Chairman_'+zeta]==1]=0
        dZdS[self.table['Chairman_'+zeta]==1]=0
        dZdpC[self.table['Chairman_'+zeta]==1]=0
        
        return(dZdaR,dZdaL,dZdaC,dZdB,dZdC,dZdMu,dZdS,dZdpC)
    
    # combined gradient
    def grad_nash(self,theta):
        
        sigma = theta[-2]
        self.get_rho_z(theta)
        
        gradaR=0.0; gradaL=0.0; gradaC=0.0; gradB=0.0; gradC=0.0; gradMu=0.0; gradS=0.0; gradpC=0.0; gradSigma=0.0
        dZ_high = np.array(self.dZ(theta,'T_high','zetaHigh'))
        dZ_low = np.array(self.dZ(theta,'T_low','zetaLow'))
        grads = np.array([gradaR, gradaL, gradaC, gradB, gradC, gradMu, gradS, gradpC, gradSigma])
        for i in range(len(grads)-1): 
            grads[i] = 1/sigma * (((self.table['zetaHigh']/sigma).apply(norm.pdf)*dZ_high[i]-(self.table['zetaLow']/sigma).apply(norm.pdf)*dZ_low[i])/\
            ((self.table['zetaHigh']/sigma).apply(norm.cdf)-(self.table['zetaLow']/sigma).apply(norm.cdf))).sum()

        grads[-1]= -1/sigma**2 * (((self.table['zetaHigh']/sigma).apply(norm.pdf)*self.table['zetaHigh']\
                                -(self.table['zetaLow']/sigma).apply(norm.pdf)*self.table['zetaLow'])/\
                                ((self.table['zetaHigh']/sigma).apply(norm.cdf)-(self.table['zetaLow']/sigma).apply(norm.cdf))).sum()
        
        return(grads)
    
    
BoC = Discrete('BoC',cpiType='TotalCPI',lam=129600,trunc=(12,None),dropCols=['TotalCPI','TotalCPI_SA','Change','Tau'])

In [9]:
theta = [ 1.68942174,  3.29229254,  4.03578095,  0.33408312, -0.86066226,
       -5.77031134,  8.65131833,  0.23652425,  1.01794943,  0.2       ]

BoC.compare_grad_to_lln(theta,epsilon=0.000000001,start=8,end=9)

Done!
You're all good!


## Grad Ascent MLE

Use the objects above to compute MLE for our model using gradient ascent. 

In [10]:
BoC.grad_ascent_nash(0.01,theta,ll=True,est=True,conv=0.0001,num_draws=10,bin_size=2*random.uniform(.25,1),pC_bin_size=0.4*random.uniform(0.25,1)) # total

[1.68942174, 3.29229254, 4.03578095, 0.33408312, -0.86066226, -5.77031134, 8.65131833, 0.23652425, 1.01794943, 0.2]
-120.08552521461363
[ 1.67043504  3.31125885  4.05478084  0.31511475 -0.87962124 -5.78931123
  8.67031809  0.2555234   1.0369386   0.2       ]
-120.03130681490936
[ 1.68457293  3.32539794  4.06941142  0.32925328 -0.8654957  -5.80370727
  8.6842391   0.24181135  1.02280137  0.2       ]
-120.03186023075432
[ 1.69328024  3.322322    4.08200318  0.33435376 -0.85233555 -5.81658485
  8.69646955  0.22884092  1.02288385  0.2       ]
-120.03028723718893
[ 1.69790893  3.31644822  4.09286301  0.33506381 -0.8568374  -5.82834667
  8.70873391  0.22695271  1.02147017  0.2       ]
-120.02541374048536
[ 1.69989267  3.30996466  4.10264101  0.3336059  -0.85449021 -5.839609
  8.72050439  0.22878489  1.0194954   0.2       ]
-120.02216204307045
[ 1.70094652  3.30397909  4.11158409  0.33180482 -0.8576506  -5.85054914
  8.73190786  0.23265823  1.01778029  0.2       ]
-120.0213018183232
[ 1.70161

-120.0088300421956
[ 1.69007904  3.29433109  4.24603595  0.33364021 -0.86070895 -6.36537754
  9.24499024  0.23589641  1.01830735  0.2       ]
-120.00861539485759
[ 1.69009523  3.29440964  4.24671475  0.33362371 -0.86071171 -6.37492649
  9.2544946   0.2358788   1.01832338  0.2       ]
-120.00840145350789
[ 1.69011208  3.29447806  4.24738069  0.33360885 -0.86071398 -6.38446577
  9.26398852  0.23586145  1.01833671  0.2       ]
-120.00818819585957
[ 1.69013055  3.29453493  4.24803462  0.33359568 -0.86071566 -6.39399543
  9.27347203  0.23584438  1.01834675  0.2       ]
-120.00797560407375
[ 1.69015077  3.29457998  4.2486773   0.33358423 -0.86071673 -6.40351549
  9.28294517  0.23582762  1.0183534   0.2       ]
-120.00776366770693
[ 1.69017212  3.29461385  4.24930941  0.33357449 -0.86071724 -6.41302598
  9.29240796  0.23581114  1.01835698  0.2       ]
-120.007552382843
[ 1.69019351  3.29463792  4.24993159  0.33356648 -0.86071732 -6.42252692
  9.30186042  0.23579491  1.01835809  0.2       ]
-1

-119.99739183158313
[ 1.6895756   3.29464293  4.27513707  0.33373361 -0.86076043 -6.90409249
  9.77994539  0.23504386  1.01852882  0.2       ]
-119.997210967823
[ 1.68956763  3.29464728  4.27553814  0.33373494 -0.86075514 -6.91312688
  9.78889529  0.2350299   1.01853207  0.2       ]
-119.99703060573698
[ 1.68956005  3.29465171  4.27593697  0.33373691 -0.86076762 -6.92215317
  9.79783654  0.23501877  1.01853557  0.2       ]
-119.9968507439526
[ 1.68955106  3.29465527  4.27633358  0.3337379  -0.86074541 -6.93117148
  9.80676896  0.23500229  1.01853855  0.2       ]
-119.99667138397822
[ 1.68954418  3.29465985  4.27672799  0.33374106 -0.86079146 -6.94018166
  9.815693    0.23499648  1.01854255  0.2       ]
-119.99649253938185
[ 1.68953246  3.29466172  4.27712023  0.33374017 -0.86070213 -6.94918409
  9.82460783  0.23496966  1.01854452  0.2       ]
-119.99631427259227
[ 1.68952981  3.29466837  4.27751031  0.33374761 -0.86088348 -6.95817808
  9.8335152   0.23498502  1.01855052  0.2       ]
-1

array([ 1.68933259,  3.29460146,  4.27942931,  0.333624  , -0.85698754,
       -7.00303878,  9.8779014 ,  0.23422615,  1.01850098,  0.2       ])

## Z-plots

Plot the distrubtion of zeta for the Fed and the Bank of Canada.

In [ ]:
plt.hist(Fed.table['nashZ'], bins=8)

In [ ]:
plt.hist(BoC.table['nashZ'], bins=8)

## Results

Estimated Coefficients for different banks, using different measures of CPI.

| Bank | $a_R$ | $a_L$ | $a_C$ | $b$ | $c$ | $\mu$ | $s$ | $p_C$ | $\sigma$ | $\epsilon$ |
|---|---|---|---|---|---|---|---|---|---|---|
| Bank of Canada Total SA|1.63409959|3.16218375|2.37604565|0.37368738|-0.81679396|1.13783871|3.74532325|1.0|0.78785134|0.2|
| Bank of Canada Total |1.74815774|3.26906496|2.45751539|0.33224235|-0.82289919|1.75681306|2.11248513|1.0|0.78725741|0.2|
| Bank of Canada Core CPI |1.80539449|3.84184997|2.7800699|0.19361319|-0.89938782|1.9975356|2.37650492|1.0|1.04782968|0.2|
| Fed |0.96405556|3.72723961|2.35336394|0.6964615|-2.69918336|-0.08721112|2.84102282|1.0|1.02183077|0.2|
|BoC Discrete Total |1.68744793|3.29453371|-1.09243813|0.33595219|-0.860996|9.17082977|7.95217247|0.41736977|1.0188812 |0.2|
